#### Day 9 - B
Decode line of block files, space between them and IDs for each block file.

File blocks have to be compressed together.

Compress the decoded line and get the sum of products of the block id and index.

In [1]:
#Import Libraries and settings

settings = {
    "day": 9,
    "test_data": 0
}

In [2]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        line = f.read()

    pairs = []
    for pair in range(int(len(line)/2)):
        pairs.append((line[pair*2], line[pair*2+1]))
    pairs.append((line[-1], 0))

    return pairs

line_raw = load_input(settings)

In [3]:
#Decode the line into pairs of characters and lengths
def decode_line_2(line):
    decoded_line = []
    for idx, pair in enumerate(line):
        #Convert to UTC so block index can be represented as single characters
        decoded_line += [(chr(idx+65), int(pair[0]))]
        decoded_line += [(".", int(pair[1]))]

    return decoded_line

line_decoded_2 = decode_line_2(line_raw)

In [4]:
line_decoded_2[0:5]

[('A', 3), ('.', 7), ('B', 1), ('.', 7), ('C', 7)]

In [5]:
[x for x in line_decoded_2[::-1] if x[0] != "."][0:5]

[('❐', 7), ('❏', 3), ('❎', 7), ('❍', 4), ('❌', 6)]

In [6]:
#Compress the line by filling the gaps between files
def compress_line_2(line):
    back_cands = [x for x in line_decoded_2[::-1] if x[0] != "."]

    #Attempt to insert the rightmost blocks in the list exactly once
    for idx, cand in enumerate(back_cands):

        #Get orginal cand index
        old_idx = line.index(cand)

        #Find space earlier in the list that fits the block
        space_idx = -1
        for idx, space in enumerate(line[:old_idx]):
            #Check if it can be substituted
            if space[0] == "." and space[1] >= cand[1]:
                space_idx = idx
                break
        
        #If no space is found, do not attempt to move the block
        if space_idx != -1:

            #Insert the block to the identified space
            line.insert(idx, cand)

            #Recalculate the empty space before and after the newly inserted block
            remainder = space[1] - cand[1]
            #Easier to 0 fill space before the inserted block as some code relies on the character being surrounded by "." blocks
            line.insert(idx, (".", 0))
            line[idx + 2] = (".", remainder)

            #Derrive the index of the original instance of the candidate block
            adj_idx = old_idx + 2

            #Remove old instance of cand
            del line[adj_idx]

            #Combine surrounding empty spaces into a single block
            if line[adj_idx-1][0] == "." and line[adj_idx][0] == ".":
                line[adj_idx-1] = (".", line[adj_idx-1][1] + line[adj_idx][1] + cand[1])
                del line[adj_idx]

    #Trim 0 width "." blocks
    line = [x for x in line if x[1] > 0]
    return line   

line_decoded_2 = decode_line_2(line_raw)
line_out = compress_line_2(line_decoded_2)

In [7]:
#Get the output from the compressed line
def calc_out(line_out):
    idx = 0
    subtotal = 0
    for pair in line_out:
        if pair[0] != ".":
            num = ord(pair[0])-65
            for i in range(pair[1]):
                subtotal += (idx + i) * num
        idx += pair[1]

    return subtotal
print(calc_out(line_out))

6448168620520
